In [2]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import pandas as pd
import numpy as np
import torchvision
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
import sklearn
from itertools import permutations
from pprint import pprint
from PIL import Image
import PIL
import pyodbc
from tqdm import tqdm
from random import random
from random import randrange
import math
import seaborn as sns
from matplotlib.lines import Line2D
from torchsummary import summary


In [ ]:
print(torch.cuda.is_available())

In [2]:
torch.cuda.set_device(0)
plt.ion()   # interactive mode
testimages = list()
testclasses = list()
testindices = list()
testinitialized = 0
PATH = './test_net4.tar'
ndepth = 4
nminibatchsize = 16
classes = ('Hospitalized','Intubated','Deceased','Pneumonia')
columns = ('Male','Pregnant','Diabetes','Asthma','Immunocompromised'
            ,'Hypertension','Other Disease','Cardiovascular Disease','Obesity','Kidney Disease'
            ,'Tobacco Use','COPD')
nclasses = len(classes)
ncolumns = len(columns)
testratio = 0.4 #randomly select this proportion of the dataset to be test data

In [3]:
nfeatures = 1.0
for i in range(ndepth-1):
    nfeatures *= (ncolumns - i)
nimgsize = math.ceil(nfeatures**0.5)
npix = nimgsize**2
normtuple = list()
for i in range(ndepth): normtuple.extend([0.5])
normtuple = tuple(normtuple)

print('total features: %i; image size: %i x %i x %i' % (nfeatures,nimgsize,nimgsize,ndepth))
print('image normalization tuple:')
print(normtuple)

total features: 1320; image size: 37 x 37 x 4
image normalization tuple:
(0.5, 0.5, 0.5, 0.5)


In [4]:
def getlabelval(labeldata):
    retval = np.array(list(labeldata),dtype=np.float32)
    return retval

def createimagetensor(row=None):
    if row:
        icol = 0
        irow = 0
        if len(list(row[0])) != ncolumns: raise CustomError('Invalid Input Size: Expected %i, Received %i' % (ncolumns,len(list(row[0]))))
        if len(row) < 2: raise CustomError('Invalid Data: Expected row format is [bindata,age,labels(optional)]')
        imgdata = np.zeros((nimgsize,nimgsize,ndepth),dtype=np.float32)
        for px in permutations(list(row[0]),ndepth-1):
            thispx = list()
            for chan in px: thispx.extend([float(chan)])
            thispx.extend([float(row[1])/100.])
            imgdata[icol,irow,:] = thispx
            irow += 1
            if (irow > nimgsize - 1):
                icol += 1
                irow = 0
        return TF.normalize(TF.to_tensor(imgdata),normtuple,normtuple)
    else:
        return None

class CustomError(Exception):
    pass


In [5]:
#create generator function for data acquisition and conversion to torch image format.
#function must return a tuple of form (tensor containing 4 images,tensor containing 4 labels)
#also builds a test set using a quick and dirty random number method.  this will need to be 
#improved to balance both the test and train sets across potential labels.

def nextrow():
    conn = pyodbc.connect('DSN=covid;UID=seh;PWD=Welcome2020!;')
    crsr = conn.cursor()
    crsr.execute("{CALL getpydatav2}")
    rowcnt = 0
    imgs = list()
    labels = list()
    row = [0,1,2]
    idx = 0
    while row:
        while (rowcnt<nminibatchsize):
            if testinitialized == 1 and idx in testindices:
                crsr.skip(nminibatchsize)
                row = [0,1,2]
                break
            else:
                try:
                    row = crsr.fetchone()
                except:
                    conn = pyodbc.connect('DSN=covid;UID=seh;PWD=Welcome2020!;')
                    crsr = conn.cursor()
                    crsr.execute("{CALL getpydatav2}")
                    crsr.skip(idx*nminibatchsize + rowcnt)
                    row = crsr.fetchone()
                rowcnt += 1
                if row:
                    imgtensor = createimagetensor(row)
                    labels.append(np.array(list(row[2]),dtype=np.float32))
                    imgs.append(imgtensor)
                else:
                    break
        if row:
            if (testinitialized == 0):
                if (random() < testratio):
                    testimages.append(torch.stack(imgs))
                    testclasses.append(torch.tensor(labels))
                    testindices.append(idx)
                else:
                    yield torch.stack(imgs),torch.tensor(labels)
            elif (idx not in testindices):
                yield torch.stack(imgs),torch.tensor(labels)
            rowcnt = 0
            idx += 1
            imgs = list()
            labels = list()
        else:
            break


In [6]:
#create generator function for test set
def gentestdata(startindex = 0):
    for i,clss in enumerate(testclasses[startindex:],startindex):
        yield testimages[i],clss

def gettestdata(idx = 0):
    return testimages[idx],testclasses[idx]

def createfakedata(age=None, conds=None):
    if age is None: age = randrange(15,90)
    elif isinstance(age,tuple): age = randrange(age[0],age[1])
    bindata = ''
    if isinstance(conds,list):
        for col in columns:
            bindata += ('1' if col in conds else '0') 
        return createimagetensor([bindata,age]).unsqueeze(0)
    elif isinstance(conds,str):
        for col in columns:
            bindata += ('1' if col in conds else '0') 
        return createimagetensor([bindata,age]).unsqueeze(0)
    totconds = (conds if isinstance(conds,int) and conds < ncolumns else math.floor(ncolumns*random()))
    condsapplied = 0
    for i,col in enumerate(columns):
        if condsapplied < totconds:
            if random() >= 0.5:
                if i == 1 and (age < 12 or age > 55 or bindata[0] == '1'):
                    bindata += '0'
                else:
                    bindata += '1'
                    condsapplied += 1
            else:
                bindata += '0'
        else:
            bindata += '0'
    return createimagetensor([bindata,age]).unsqueeze(0)

In [7]:

class Net(nn.Module):

    def __init__(self, modelinputtensor, layerlist, layerparams):
        """
        torch.nn must be imported as nn
        torch.nn.functional must be imported as F
        modelinputtensor = example model input tensor (including an arbitrary batch dimension)
        layerlist = list of pytorch nn fucntions as their 'F' namespace equivalents (e.g. 'nn.MaxPool2d' should be supplied as 'F.max_pool2d') 
        layerparams = list of _independent_ params in their nn form and passed as a tuple.  Example:
            The first conv2d layer will have 3 params in a tuple of form (in_channels, out_channels, kernel_size). 
            Subsequent conv2d layers will have _2_ params in a tuple of form (out_channels, kernel_size) since the in_channels will be determined by the previous layer. 
            Pooling layers will always have params of the form (x, y) corresponding to the pooling window size.
            Linear layers will always have a single param corresponding to the number of out features for the layer (input features are determined by the preceding layer)
        """
        super(Net, self).__init__()
        self.lyrs, self.fwdlyrs = self.getlayers(modelinputtensor,layerlist,layerparams)

    def forward(self, x):
        for f in self.fwdlyrs:
            x = eval(f)
        return F.sigmoid(x)

    def getlayers(self,testtensor,funcs=[],params=[],debug=0):
        initlayers = nn.ModuleList()
        fwdlayers = list()
        if debug == 1: print(testtensor.size())
        lastsize = None
        lyr = 0
        with torch.no_grad():
            for fn,pa in zip(funcs,params):
                if lastsize is not None:
                    if fn.__name__ == 'conv2d':
                        pa = (lastsize[1],pa[0],pa[1])
                    elif fn.__name__ == 'linear':
                        if not testtensor.ndim == 2:
                            testtensor = testtensor.view(-1,self.numflatfeatures(testtensor))
                            fwdlayers.append("x.view(-1,self.numflatfeatures(x))")
                            lastsize = testtensor.size()
                        pa = (lastsize[1],pa)        
                if fn.__name__ == 'conv2d':
                    paeval = "torch.tensor(np.random.rand(" + ",".join(tuple(map(str,(pa[1],pa[0],pa[2],pa[2])))) + "),dtype=torch.float32)"
                elif fn.__name__ == 'max_pool2d':
                    paeval = ",".join(tuple(map(str,pa)))
                elif fn.__name__ == 'linear':
                    paeval = "torch.tensor(np.random.rand(" + ",".join(tuple(map(str,(pa[1],pa[0])))) + "),dtype=torch.float32)"
                if not fn.__name__ == 'linear' or pa[0] > pa[1]:
                    testtensor = fn(testtensor,eval(paeval))
                    lastsize = testtensor.size()
                    initlayers.append(eval(self.getinitequivalent(fn.__name__,pa)))
                    fwdlayers.append(self.getfwdequivalent(fn.__name__,lyr))
                    lyr += 1
                    if debug == 1: print(testtensor.size())
                elif debug == 1: print('Eliminating linear layer - out features > previous layer')
        fwdlayers[-1] = 'self.lyrs[' + str(lyr - 1) + '](x)'
        return initlayers,fwdlayers

    def numflatfeatures(self,x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

    def getinitequivalent(self,funcname,initparams):
        return 'nn.' + ''.join([val.capitalize() for val in funcname.split('_')]) + '(' + ",".join(tuple(map(str,initparams))) + ')'

    def getfwdequivalent(self,funcname,lyrnum):
        if not funcname == 'max_pool2d':
            return 'F.relu(self.lyrs[' + str(lyrnum) + '](x))'
        else:
            return 'self.lyrs[' + str(lyrnum) + '](x)'




In [8]:
def getconvlayers(firstdepth=4,maxlayers=5,minlayers=1,maxkernel=7,minkernel=3,maxoutchan=12,minoutchan=4):
    fncs,parms = list(),list()
    fncs.append(F.conv2d)
    parms.append((firstdepth,np.random.randint(minoutchan,maxoutchan+1),np.random.randint(minkernel,maxkernel+1)))
    for i in range(np.random.randint(minlayers-1,maxlayers)):
        fncs.append(F.conv2d)
        parms.append((np.random.randint(minoutchan,maxoutchan+1),np.random.randint(minkernel,maxkernel+1)))
    return fncs,parms

def getlinlayers(initoutfeatures,featuredeadband=20,maxlayerdivisor=20,minlayerdivisor=4):
    fncs,parms = list(),list()
    fncs.append(F.linear)
    parms.append(initoutfeatures)
    nextoutfeatures = int(initoutfeatures/np.random.randint(minlayerdivisor,maxlayerdivisor+1)) 
    while nextoutfeatures > nclasses + featuredeadband:
        fncs.append(F.linear)
        parms.append(nextoutfeatures)
        nextoutfeatures = int(nextoutfeatures/np.random.randint(minlayerdivisor,maxlayerdivisor+1))
    fncs.append(F.linear)
    parms.append(nclasses)
    return fncs,parms

In [9]:
datlist = []
for i in range(4):
    datlist.append(createfakedata(40,["Diabetes"]).squeeze(0))
testtensor = torch.stack(datlist)
optimizers = [("optim.SGD(d['net'].parameters(), lr=0.0001, momentum=0.9)","SGD lr=1 p=.9"),
              ("optim.Adam(d['net'].parameters(), lr=0.0001)","lr=1 Adam"),
              ("optim.Adam(d['net'].parameters(), lr=0.00001)","lr=0.1 Adam"),
             ]
def createtestnets(netcount=50,**kwargs):
    netdict = dict()
    for i in tqdm(range(netcount)):
        cfs, cps = getconvlayers()
        lfs, lps = getlinlayers(1000)
        funcs = cfs
        params = cps
        if random() > 0.3:
            funcs.extend([F.max_pool2d])
            poolsize = np.random.randint(2,4)
            params.extend([(poolsize,poolsize)])
        funcs.extend(lfs)
        params.extend(lps)
        for opt in optimizers:
            d = dict()
            d['net'] = Net(testtensor,funcs,params)
            d['criterion'] = nn.BCELoss()
            d['optimizer'] = eval(opt[0])
            #d['net'].cuda()
            netdict[str(i) + '-' + opt[1]] = d
    return netdict

netcount = 100
netdict = createtestnets(netcount)
funcs = [F.conv2d,F.conv2d,F.max_pool2d,F.linear,F.linear,F.linear]
params = [(4,12,5),(48,5),(2,2),578,34,4]
for opt in optimizers:
    d = dict()
    d['net'] = Net(testtensor,funcs,params)
    d['criterion'] = nn.BCELoss()
    d['optimizer'] = eval(opt[0])#optim.SGD(d['net'].parameters(), lr=0.0001, momentum=0.9)
    #d['net'].cuda()
#print(summary(d['net'],(ndepth,nimgsize,nimgsize)))
    netdict[str(netcount) + '-' + opt[1]] = d
        

100%|██████████| 100/100 [00:10<00:00,  9.30it/s]


In [10]:
for n in range(netcount+1):
    for o in optimizers:
        print('Net number ' + str(n))
        print(summary(netdict[str(n) + '-' + o[1]]['net'],(ndepth,nimgsize,nimgsize)))
        print()
        print()
        break

Net number 0


RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [11]:
checkpoint = torch.load('./train_test_data.tar')
train_data = checkpoint['train_data']
testimages = checkpoint['testimages']
testindices = checkpoint['testindices']
testclasses = checkpoint['testclasses']

In [3]:
checkpoint = torch.load('./train_test_data.tar')
train_data = checkpoint['train_data']
testimages = checkpoint['testimages']
testindices = checkpoint['testindices']
testclasses = checkpoint['testclasses']
torch.save({'train_data':train_data,
            'validation_images':testimages,
            'validation_indices': testindices,
            'validation_labels': testclasses,
            },'./datasets.tar')

In [ ]:
train_data = list()
rowgen = nextrow()
pbar = tqdm(enumerate(rowgen),total=1200)
for i, data in pbar:
    train_data.append(data)
testinitialized = 1
torch.save({'train_data':train_data,
            'testimages':testimages,
            'testindices': testindices,
            'testclasses': testclasses,
            },'./train_test_data.tar')

In [12]:
#train a series of layer configurations and record loss data
loss_recording_rate = 10
lossdict = dict()
#for epoch in range(2):  # loop over the dataset multiple times
for k,d in netdict.items():
    net = d['net']
    net.cuda()
    net.train()
    criterion = d['criterion']
    optimizer = d['optimizer'] 
    tlosslist = []
    vlosslist = []
    last_loss = 0.0
    lasttestidx = 0
    running_loss = 0.0
    testcnt = 0
    train_cnt = len(train_data)
    validationcnt = len(testimages)
    #rowgen = nextrow()
    #pbar = tqdm(enumerate(rowgen),total=validationfreq-1)
    pbar = tqdm(enumerate(train_data),total=train_cnt)
    for i, data in pbar:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        outputs = net(inputs)
        #print((outputs,labels))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        inputs, labels = inputs.to('cpu'), labels.to('cpu')
        if i % loss_recording_rate == loss_recording_rate - 1:
            tlosslist.append((running_loss-last_loss)/loss_recording_rate)
            pbar.set_description(desc='net name: %s; loss: %.3f' % (k, running_loss/(i+1)))
            pbar.update()
            last_loss = running_loss
    last_loss = 0.0
    valid_loss = 0.0
    net.eval()
    with torch.no_grad():
        randomstartindex = 0#math.floor(len(testimages)*random())
        genfunc = gentestdata(randomstartindex)
        pbar = tqdm(enumerate(genfunc),total=validationcnt)
        for j, (tin,tlab) in pbar:
            tin, tlab = tin.cuda(), tlab.cuda()
            outputs = net(tin)
            loss = criterion(outputs,tlab)
            valid_loss += loss.item()
            if j % loss_recording_rate == loss_recording_rate - 1:
                vlosslist.append((valid_loss-last_loss)/loss_recording_rate)
                last_loss = valid_loss
                pbar.set_description(desc='net name: %s; loss: %.3f; validation loss: %.3f' 
                                        % (k, running_loss/train_cnt, valid_loss/(j+1)))
                pbar.update()
    #print('Training Complete for %s; loss: %.3f; validation loss: %.3f' % (k, running_loss/train_cnt, valid_loss/validationcnt))

    lossdict[k] = {'trainlosses':tlosslist,
                    'validlosses': vlosslist,
                    'trainlossavg': running_loss / train_cnt,
                    'validlossavg': valid_loss / validationcnt}
    net.cpu()
    torch.cuda.empty_cache()
    running_loss = 0.0
    last_loss = 0.0
#save test set and model state dictionary for loading later without training.
#torch.save({'model_state_dict':net.state_dict(),
#            'testimages': testimages,
#            'testclasses': testclasses,
#            'testindices': testindices}, PATH)
print('Finished Training')

net name: 0-SGD lr=1 p=.9; loss: 0.663: 100%|██████████| 1290/1290 [00:04<00:00, 261.39it/s]
net name: 0-SGD lr=1 p=.9; loss: 0.663; validation loss: 0.642: 100%|██████████| 834/834 [00:01<00:00, 609.35it/s]
net name: 0-lr=1 Adam; loss: 0.469: 100%|██████████| 1290/1290 [00:03<00:00, 326.93it/s]
net name: 0-lr=1 Adam; loss: 0.469; validation loss: 0.443: 100%|██████████| 834/834 [00:00<00:00, 861.95it/s]
net name: 0-lr=0.1 Adam; loss: 0.542: 100%|██████████| 1290/1290 [00:03<00:00, 329.91it/s]
net name: 0-lr=0.1 Adam; loss: 0.542; validation loss: 0.469: 100%|██████████| 834/834 [00:00<00:00, 860.21it/s]
net name: 1-SGD lr=1 p=.9; loss: 0.653: 100%|██████████| 1290/1290 [00:03<00:00, 387.08it/s]
net name: 1-SGD lr=1 p=.9; loss: 0.653; validation loss: 0.637: 100%|██████████| 834/834 [00:00<00:00, 895.61it/s]
net name: 1-lr=1 Adam; loss: 0.478: 100%|██████████| 1290/1290 [00:03<00:00, 334.36it/s]
net name: 1-lr=1 Adam; loss: 0.478; validation loss: 0.446: 100%|██████████| 834/834 [00:00

KeyboardInterrupt: 

In [ ]:
for k,d in netdict.items():
    d['net'].cpu()

In [ ]:
#print(goodnets[0]['net'].lyrs)
netdf = pd.DataFrame(columns=['netnum','iteration','loss','losstype'])
loc = 0
for k,gnet in netdict.items():
    for i,loss in enumerate(lossdict[k]['trainlosses']):
        netdf.loc[loc] = pd.Series({'netnum':k,'iteration':i,'loss':loss,'losstype':'train'})
        loc += 1
    for i,loss in enumerate(lossdict[k]['validlosses']):
        netdf.loc[loc] = pd.Series({'netnum':k,'iteration':i,'loss':loss,'losstype':'test'})
        loc += 1
#agedf.loc[0] = pd.Series({'age':0,'type':'Predicted','result':'Deceased'})
#print(netdf.iloc[0:50])

In [ ]:
fig, axs = plt.subplots(netcount+1,1,figsize=(20,25))
for i,ax in enumerate(axs):
    sns.lineplot(ax=ax,x='iteration', y='loss', hue='netnum', style='losstype', data=netdf[netdf.netnum.str.startswith(str(i))])


In [ ]:
net_summary_df = pd.DataFrame(columns=['net_number',
                                       'param_count',
                                       'conv_layers',
                                       'has_pool',
                                       'lin_layers',
                                       'max_average_test_loss',
                                       'max_loss_optimizer',
                                       'max_loss_optimizer_efficiency',
                                       'min_average_test_loss',
                                       'min_loss_optimizer',
                                       'min_loss_optimizer',
                                       'min_loss_optimizer_efficiency'])
last_net_num = 0
param_count = 0
conv_layers = 0
has_pool = 0
lin_layers = 0
max_loss_optimizer = ''
min_loss_optimizer = ''
max_average_test_loss = 0.0
min_average_test_loss = 1.0
optimizer_type = ''
loc = 0
net_d = dict()
for k,net_d in netdict.items():
    net_number = int(k[:k.find('-')])
    optimizer_type = k[k.find('-')+1:]
    if last_net_num != net_number:
        param_count = sum(p.numel() for p in net_d['net'].parameters() if p.requires_grad)
        conv_layers = 0
        has_pool = 0
        lin_layers = 0
        for lyr in net_d['net'].lyrs:
            if str(lyr).startswith('Conv2d'):
                conv_layers = conv_layers + 1 
            if str(lyr).startswith('MaxPool'):
                has_pool = 1 
            if str(lyr).startswith('Linear'):
                lin_layers = lin_layers + 1 
        net_summary_df.loc[loc] = pd.Series({'net_number':last_net_num,
                                             'param_count':param_count,
                                             'conv_layers':conv_layers,
                                             'has_pool':has_pool,
                                             'lin_layers':lin_layers,
                                             'max_average_test_loss':max_average_test_loss,
                                             'max_loss_optimizer':max_loss_optimizer,
                                             'max_loss_optimizer_efficiency':(1/(param_count*max_average_test_loss))*10000.0,
                                             'min_average_test_loss':min_average_test_loss,
                                             'min_loss_optimizer':min_loss_optimizer,
                                             'min_loss_optimizer_efficiency':(1/(param_count*min_average_test_loss))*10000.0,
                                             })
        loc += 1
        max_loss_optimizer = ''
        min_loss_optimizer = ''
        max_average_test_loss = 0.0
        min_average_test_loss = 1.0
    average_test_loss = netdf[netdf.netnum==k][netdf.losstype=='test'].loss.mean()
    if average_test_loss < min_average_test_loss:
        min_average_test_loss = average_test_loss
        min_loss_optimizer = optimizer_type
    if average_test_loss > max_average_test_loss:
        max_average_test_loss = average_test_loss
        max_loss_optimizer = optimizer_type
    last_net_num = net_number

param_count = sum(p.numel() for p in net_d['net'].parameters() if p.requires_grad)
conv_layers = 0
has_pool = 0
lin_layers = 0
for lyr in net_d['net'].lyrs:
    if str(lyr).startswith('Conv2d'):
        conv_layers = conv_layers + 1 
    if str(lyr).startswith('MaxPool'):
        has_pool = 1 
    if str(lyr).startswith('Linear'):
        lin_layers = lin_layers + 1 
net_summary_df.loc[loc] = pd.Series({'net_number':net_number,
                                        'param_count':param_count,
                                        'conv_layers':conv_layers,
                                        'has_pool':has_pool,
                                        'lin_layers':lin_layers,
                                        'max_average_test_loss':max_average_test_loss,
                                        'max_loss_optimizer':max_loss_optimizer,
                                        'max_loss_optimizer_efficiency':(1/(param_count*max_average_test_loss))*10000.0,
                                        'min_average_test_loss':min_average_test_loss,
                                        'min_loss_optimizer':min_loss_optimizer,
                                        'min_loss_optimizer_efficiency':(1/(param_count*min_average_test_loss))*10000.0,
                                        })

net_summary_df = net_summary_df.sort_values('max_loss_optimizer_efficiency')
net_summary_df

In [ ]:
net = netdict['0-lr=1 Adam']['net']

In [ ]:
def interpretimagetensor(imgtensor):
    ptsdata = list()
    for (bindata,age) in getbindatafromimagetensor(imgtensor):
        ptdata = str(int(age)) + ' yo, '
        for i, val in enumerate(list(bindata)):
            if i == 0:
                if val == '1':
                    ptdata += 'Male, '
                else:
                    ptdata += 'Female, '
            elif val == '1':
                ptdata += columns[i] + ', '
        ptdata = ptdata[:-2]
        ptsdata.append(ptdata)
    return ptsdata

def getbindatafromimagetensor(imgtensor):
    ptsdata = list()
    for case in imgtensor:
        case = case / 2 + 0.5
        case = case.numpy()
        case = np.transpose(case, (1, 2, 0))
        bindata = ''
        for i in range(ndepth-1): bindata += str(int(case[0,0,i]))
        for i in range(1,ncolumns - ndepth + 2):
            bindata += str(int(case[0,i,ndepth-2]))
        age = float(round(float(case[0,0,ndepth-1])*100.))
        ptsdata.append((bindata, age))
    return ptsdata


In [ ]:
#skip training and load saved model
checkpoint = torch.load(PATH)
net = Net()
net.load_state_dict(checkpoint['model_state_dict'])
testimages = checkpoint['testimages']
testclasses = checkpoint['testclasses']
testindices = checkpoint['testindices']
testinitialized = 1
net.eval()

In [ ]:
#test random image from test set
net.eval()
net.cpu()
randomstartindex = math.floor(len(testimages)*random())
print(randomstartindex)
images, labels = gettestdata(randomstartindex)

with torch.no_grad():
    outputs = net(images)
    ptinfo = interpretimagetensor(images)
    for i in range(nminibatchsize):
        actlist = list()
        predlist = list()
        if sum(labels[i]) == 0.:
            actlist.append('none')
        for j in range(nclasses):
            predlist.extend([str(round(100*float(outputs[i,j]),2)) + '% ' + classes[j]])
            if labels[i,j] == 1.: actlist.extend([classes[j]])
        print(ptinfo[i])
        print(predlist)
        print(actlist)
        print()

In [ ]:
fakedata = createfakedata(age=(51,61),conds=['Male','Diabetes','Hypertension'])
outputs = net(fakedata)
print(interpretimagetensor(fakedata))
predlist = list()
for i in range(nclasses):
    predlist.extend([str(round(100*float(outputs[0,i]),2)) + '% ' + classes[i]])
print(predlist)


In [ ]:
labelstats = dict()
for c in classes:
    labelstats[c] = {
        "predprobs" : list(),
        "age" : list(),
        "sex" : list(),
        "conditions" : list()
    }

with torch.no_grad():
    genfunc = gentestdata(0)
    for j, (images,labels) in tqdm(enumerate(genfunc,0),total=len(testimages)):
        outputs = net(images)
        outcomes = np.array(outputs)
        ptinfo = interpretimagetensor(images)
        for i, case in enumerate(outcomes):
            for k, lab in enumerate(labels[i]):
                if lab == 1:
                    thisdict = labelstats[classes[k]]
                    thisdict['predprobs'].append(list(case))
                    thisinfo = ptinfo[i].split(', ')
                    thisdict['age'].append(int(thisinfo[0].strip(' yo')))
                    thisdict['sex'].append(thisinfo[1])
                    thisdict['conditions'].append(thisinfo[2:])



In [ ]:
flatlblstats = {
    "truth" : list(),
    "predprobs" : list(),
    "age" : list(),
    "conditions" : list()
    }

with torch.no_grad():
    genfunc = gentestdata(0)
    for j, (images,labels) in tqdm(enumerate(genfunc,0),total=len(testimages)):
        outputs = net(images)
        outcomes = np.array(outputs)
        ptinfo = interpretimagetensor(images)
        for i, case in enumerate(outcomes):
            flatlblstats['truth'].append([classes[j] for (j,lab) in enumerate(labels[i]) if lab == 1])
            flatlblstats['predprobs'].append(list(case))
            thisinfo = ptinfo[i].split(', ')
            flatlblstats['age'].append(int(thisinfo[0].strip(' yo')))
            if thisinfo[1] == 'Male':
                flatlblstats['conditions'].append(thisinfo[1:])
            else:
                flatlblstats['conditions'].append(thisinfo[2:])



In [ ]:
predsbyage = {
    'age': list(),
    }
actualbyage = {
    'age': list(),
    }
for c in classes:
    predsbyage[c] = list()
    actualbyage[c] = list()

for i in tqdm(range(100)):
    predsums = np.zeros(nclasses,dtype=np.float32)
    actsums = np.zeros(nclasses,dtype=np.float32)
    #actsums *= 10
    for j,ag in enumerate(flatlblstats['age']):
        if ag == i:
            for k, val in enumerate(flatlblstats['predprobs'][j]):
                predsums[k] += val
            for k, val in enumerate(flatlblstats['truth'][j]):
                actsums[classes.index(val)] += 1
    predsbyage['age'].append(i)
    actualbyage['age'].append(i)
    for j, c in enumerate(classes):
        predsbyage[c].append(predsums[j])
        actualbyage[c].append(actsums[j])



In [ ]:
agedistfromactual = dict()
agedistfrompreds = dict()
for c in classes:
    agedistfromactual[c] = list()
    agedistfrompreds[c] = list()
    for i,val in enumerate(actualbyage[c]):
        agedistfromactual[c].extend([i]*int(val))
    for i,val in enumerate(predsbyage[c]):
        agedistfrompreds[c].extend([i]*int(round(val)))


In [ ]:
agedf = pd.DataFrame(columns=['age','type','result'])
#agedf.loc[0] = pd.Series({'age':0,'type':'Predicted','result':'Deceased'})
runi = 0
for c in classes:
    print('Begin predicted ' + c + ' - %i total records' % (runi))
    for i,val in enumerate(agedistfrompreds[c]):
        agedf.loc[i+runi] = pd.Series({'age':float(val),'type':'Predicted','result':c})
    runi += (i + 1)
    print('Begin actual ' + c + ' - %i total records' % (runi))
    for i,val in enumerate(agedistfromactual[c]):
        agedf.loc[i+runi] = pd.Series({'age':float(val),'type':'Actual','result':c})
    runi += (i + 1)

print('Complete - %i total records' % (runi))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,8))
ax = sns.violinplot(x='result',y='age',hue='type',data=agedf,split=True,inner='quartile')
ax.set_title('Actual vs. Predicted Age Distribution by Outcome',fontsize=16)


In [ ]:
def getactavgsbycondfromdb(condlist, includeSex = 0, minage = 0, maxage = 100):
    conn = pyodbc.connect('DSN=covid;UID=seh;PWD=Welcome2020!;')
    bindata = ''
    for col in columns[1:]:
        bindata += '1' if col in condlist else '0'
    df = pd.read_sql_query("{CALL getLabelsForConditionSetv2 ('" + bindata + "'," + str(includeSex) + "," + str(minage) + "," + str(maxage) + ")}",conn)
    return df

In [ ]:
def getstandarderror(df=pd.DataFrame):
    errdf = pd.DataFrame(columns=list(df.axes[1])[1:-1]).add_suffix('_Error')
    loc = 0
    for r in df.iloc:
        errs = list()
        N = r['CaseCount']
        for c in classes:
            errs.append((((int(r[c]*N)*(r[c]-1)**2 + int((1-r[c])*N)*r[c]**2)/(N-1))**0.5)/N**0.5)
        errdf.loc[loc] = pd.Series({'Hospitalized_Error':errs[0],'Intubated_Error':errs[1],'Deceased_Error':errs[2],'Pneumonia_Error':errs[3]})
        loc += 1
    return errdf

In [ ]:
def getconditioncompplotdata(conds1=[],conds2=[],miage=0,maage=100,micnt=0):
    agesexconddf = pd.DataFrame(columns=['Age','Sex','Condition','Hospitalized','Intubated','Deceased','Pneumonia'])
    mconds1 = list()
    mconds2 = list()
    if len(conds1) == 0: conds1.append('None')
    if len(conds2) == 0: conds2.append('None')
    mconds1 = conds1.copy()
    mconds1.append('Male')
    mconds2 = conds2.copy()
    mconds2.append('Male')
    loc = 0
    actavgs1 = getactavgsbycondfromdb(conds1,0,miage,maage)
    actavgs1 = actavgs1.join(getstandarderror(actavgs1))
    actavgs1 = actavgs1.loc[actavgs1['CaseCount'] >= micnt]
    actavgs2 = getactavgsbycondfromdb(conds2,0,miage,maage)
    actavgs2 = actavgs2.join(getstandarderror(actavgs2))
    actavgs2 = actavgs2.loc[actavgs2['CaseCount'] >= micnt]
    with torch.no_grad():
        for a in tqdm(range(miage,maage)):
            mdata = np.array(net(createfakedata(age=a,conds=mconds1)))[0]
            fdata = np.array(net(createfakedata(age=a,conds=conds1)))[0]
            mdata2 = np.array(net(createfakedata(age=a,conds=mconds2)))[0]
            fdata2 = np.array(net(createfakedata(age=a,conds=conds2)))[0]
            agesexconddf.loc[loc] = pd.Series({'Age':float(a),'Sex':'Male','Condition':', '.join(conds1),'Hospitalized':mdata[0],'Intubated':mdata[1],'Deceased':mdata[2],'Pneumonia':mdata[3]})
            loc += 1
            agesexconddf.loc[loc] = pd.Series({'Age':float(a),'Sex':'Female','Condition':', '.join(conds1),'Hospitalized':fdata[0],'Intubated':fdata[1],'Deceased':fdata[2],'Pneumonia':fdata[3]})
            loc += 1
            agesexconddf.loc[loc] = pd.Series({'Age':float(a),'Sex':'Male','Condition':', '.join(conds2),'Hospitalized':mdata2[0],'Intubated':mdata2[1],'Deceased':mdata2[2],'Pneumonia':mdata2[3]})
            loc += 1
            agesexconddf.loc[loc] = pd.Series({'Age':float(a),'Sex':'Female','Condition':', '.join(conds2),'Hospitalized':fdata2[0],'Intubated':fdata2[1],'Deceased':fdata2[2],'Pneumonia':fdata2[3]})
            loc += 1
    actavgs1 = actavgs1.loc[actavgs1['Age'] >= min(agesexconddf['Age'])]
    actavgs1 = actavgs1.loc[actavgs1['Age'] <= max(agesexconddf['Age'])]
    actavgs2 = actavgs2.loc[actavgs2['Age'] >= min(agesexconddf['Age'])]
    actavgs2 = actavgs2.loc[actavgs2['Age'] <= max(agesexconddf['Age'])]
    return agesexconddf,actavgs1,actavgs2


In [ ]:
agesexconddf,actavgs1,actavgs2 = getconditioncompplotdata(['Diabetes','Hypertension'],['Tobacco Use'],20,80,20)
fig, axs = plt.subplots(2,2,figsize=(15,12),sharex=True,sharey=False)
i = 0
for k, axi in enumerate(axs):
    for j in range(len(axi)):
        axe = axi[j]
        sns.lineplot(ax=axe, x='Age', y=classes[i], hue='Condition', data=agesexconddf)
        axe.errorbar(x='Age',y=classes[i],xerr=5.,yerr=classes[i]+'_Error',ls='None',color='blue',
                    data=actavgs1,marker='o')
        axe.errorbar(x='Age',y=classes[i],xerr=5.,yerr=classes[i]+'_Error',ls='None',color='orange',
                    data=actavgs2,marker='s')
        axe.set_title(classes[i])
        axe.set_ylabel('Incidence Rate')
        i += 1


In [ ]:
fix, axs = plt.subplots(2,2,figsize=(15,12),sharex=True,sharey=True)
custom_lines = [Line2D([0], [0], color='blue', lw=1),
                Line2D([0], [0], color='orange', lw=1),
                Line2D([0], [0], color='blue', lw=1, dashes=(4.,3.)),
                Line2D([0], [0], color='orange', lw=1, dashes=(4.,3.)),
                Line2D([0], [0], color='grey', lw=1, dashes=(4.,3.))]
i = 0
for k, axi in enumerate(axs):
    for j in range(2):
        axe = axi[j]
        sns.distplot(labelstats[classes[i]]['age'], hist = False, kde = True,kde_kws = {'linewidth': 1}
                        ,ax=axe,axlabel=classes[i]+' age distributions',color='blue')
        sns.distplot(agedistfrompreds[classes[i]], hist = False, kde = True,kde_kws = {'linewidth': 1}
                        ,ax=axe,color='orange')
        axe.axvline(x=round(np.mean(flatlblstats['age']),1),color='grey',dashes=(4.,3.),lw=1)
        axe.axvline(x=round(np.mean(labelstats[classes[i]]['age']),1),color='blue',dashes=(4.,3.),lw=1)
        axe.axvline(x=round(np.mean(agedistfrompreds[classes[i]]),1),color='orange',dashes=(4.,3.),lw=1)
        axe.legend(custom_lines,('actual ('+str(len(labelstats[classes[i]]['age']))+')','predicted ('+str(len(agedistfrompreds[classes[i]]))+')'
                                                        ,'actual mean ('+str(round(np.mean(labelstats[classes[i]]['age']),1))+')'
                                                        ,'predicted mean ('+str(round(np.mean(agedistfrompreds[classes[i]]),1))+')'
                                                        ,'overall mean ('+str(round(np.mean(flatlblstats['age']),1))+')'))
        i += 1

In [ ]:
sw

In [ ]:
truthcondheatmap = np.zeros((nclasses,ncolumns),dtype=np.float32)
predcondheatmap = np.zeros((ncolumns,nclasses),dtype=np.float32)
for i,trth in enumerate(flatlblstats['truth']):
    for cond in flatlblstats['conditions'][i]:
        for lab in trth:
            truthcondheatmap[classes.index(lab),columns.index(cond)] += 1
        predcondheatmap[columns.index(cond)] += flatlblstats['predprobs'][i]
predcondheatmap = predcondheatmap.T
for i in range(nclasses):
    truthcondheatmap[i] /= len(labelstats[classes[i]]['age'])
    predcondheatmap[i] /= sum(predsbyage[classes[i]])
fig, axs = plt.subplots(1,3,figsize=(20,6))
sns.heatmap(truthcondheatmap.T,ax=axs[0],yticklabels=columns,xticklabels=classes,annot=True,fmt='.2f')
sns.heatmap(predcondheatmap.T,ax=axs[1],yticklabels=False,xticklabels=classes,annot=True,fmt='.2f')
sns.heatmap((truthcondheatmap - predcondheatmap).T,ax=axs[2],yticklabels=False,xticklabels=classes,annot=True,fmt='.2f',cmap='bwr',vmin=-0.2,vmax=0.2)
fig.suptitle('Input Frequency vs...',fontsize=16)
axs[0].title.set_text('Actual Label')
axs[1].title.set_text('Normalized Label Predictions')
axs[2].title.set_text('Actual Minus Predicted (Model Error)')